資料處理-TOKYO（按日期跟航班代碼排序）

In [31]:
import os
import pandas as pd

def process_and_merge_files(start_date, end_date, base_path, output_path):
    # 建立日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2024-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/tokyo_{date}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if not df.empty:
                df = df.dropna(how="all")
                
                # 格式化出發日期並去除時間
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                
                # 新增星期欄位
                df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                
                # 移除價格中的符號並轉換為數字
                df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]  # 過濾掉不符合數字格式的行
                df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                
                # 計算 day left
                file_date = pd.to_datetime(f"2024-{date[:2]}-{date[2:]}")
                df['day left'] = (pd.to_datetime(df['出發日期']) - file_date).dt.days
                
                # 調整欄位順序，將「星期」移到「出發時間」後面
                cols = df.columns.tolist()
                cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('星期')))
                df = df[cols]
                
                data_frames.append(df)

    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    if merged_data.empty:
        print("沒有合併到任何資料。")
        return

    # 替換航空公司名稱
    airline_mapping = {
        'EVA Air': '長榮航空',
        'China Airlines': '中華航空',
        'Cathay Pacific': '國泰航空',
        'Jetstar': '捷星日本航空',
        'Tigerair Taiwan': '台灣虎航',
        'STARLUX Airlines': '星宇航空',
        'JAL': '日本航空',
        'Peach Aviation': '樂桃航空',
        'ANA': '全日空航空',
        'Thai Lion Air': '泰國獅航',
        'Scoot': '酷航'
    }
    merged_data['航空公司'] = merged_data['航空公司'].replace(airline_mapping)

    # 按條件分組（出發日期、出發時間、航班代碼、機型）
    grouped = merged_data.groupby(['出發日期', '航班代碼', '機型'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median')
    )
    grouped['平均價格'] = grouped['平均價格'].round().astype(int)

    # 計算最低價格的剩餘天數
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼', '機型'])['價格'].idxmin(),
        ['出發日期', '航班代碼', '機型', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)

    # 合併分組後的統計數據
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼', '機型'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼', '機型'], how='inner')

    # 去除重複行，只保留同一條航班的第一筆資料
    final_data = final_data.drop_duplicates(subset=['出發日期', '航班代碼', '機型'])

    # 排序結果
    final_data = final_data.sort_values(by=['出發日期', '航班代碼']).reset_index(drop=True)

    # 儲存結果
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，結果已按出發日期與航班代碼排序，儲存到 {output_path}")

# 主程式執行流程
base_path = '/Users/yuchingchen/Documents/專題/data'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/tokyo.csv'

# 執行合併與處理
process_and_merge_files('1021', '1118', base_path, output_path)

處理完成，結果已按出發日期與航班代碼排序，儲存到 /Users/yuchingchen/Documents/專題/cleaned_data/tokyo.csv


資料處理-SYDNEY（按日期跟航班代碼排序）

In [13]:
import os
import pandas as pd
import re

def process_and_merge_files(start_date, end_date, base_path, output_path):
    # 建立日期範圍
    dates = pd.date_range(start=f'2024-{start_date[:2]}-{start_date[2:]}', 
                          end=f'2024-{end_date[:2]}-{end_date[2:]}')
    date_strings = [date.strftime('%m%d') for date in dates]

    data_frames = []
    for date in date_strings:
        file_path = f'{base_path}/sydney_{date}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            if not df.empty:
                df = df.dropna(how="all")
                
                # 格式化出發日期並去除時間
                df['出發日期'] = pd.to_datetime(df['出發日期'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]).dt.strftime('%Y-%m-%d')
                
                # 新增星期欄位
                df['星期'] = pd.to_datetime(df['出發日期']).dt.day_name(locale='zh_TW')
                
                # 移除價格中的符號並轉換為數字
                df = df[df['價格'].str.match(r'^[NT\$,\d\s]+$')]  # 過濾掉不符合數字格式的行
                df['價格'] = df['價格'].replace(r'[NT\$,\s]', '', regex=True).astype(int)
                
                # 計算 day left
                file_date = pd.to_datetime(f"2024-{date[:2]}-{date[2:]}")
                df['day left'] = (pd.to_datetime(df['出發日期']) - file_date).dt.days
                
                # 調整欄位順序，將「星期」移到「出發時間」後面
                cols = df.columns.tolist()
                cols.insert(cols.index('出發時間') + 1, cols.pop(cols.index('星期')))
                df = df[cols]
                
                data_frames.append(df)

    # 合併所有資料
    merged_data = pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()
    
    if merged_data.empty:
        print("沒有合併到任何資料。")
        return

    # 替換航空公司名稱
    airline_mapping = {
        r"\bChina Airlines\b": "中華航空",
        r"\bEVA Air\b": "長榮航空",
        r"\bCathay Pacific\b": "國泰航空",
        r"\bQantas\b": "澳洲航空",
        r"\bChina Southern\b": "南方航空",
        r"\bChina Eastern\b": "東方航空",
        r"\bSingapore Airlines\b": "新加坡航空",
        r"\bKorean Air\b": "大韓航空",
        r"\bAsiana\b": "韓亞航空",
        r"\bANA\b": "全日空",
        r"\bJAL\b": "日本航空",
        r"\bAirAsia X\b": "亞洲航空",
        r"\bJuneyao Airlines\b": "吉祥航空",
        r"\bPhilippine Airlines\b": "菲律賓航空",
        r"\bVietnam Airlines\b": "越南航空",
        r"\bMalaysia Airlines\b": "馬來西亞航空",
        r"\bThai\b": "泰國航空",
        r"\bScoot\b": "酷航",
        r"\bXiamenAir\b": "廈門航空",
        r"\bGaruda Indonesia\b": "印尼嘉魯達航空",
        r"\bVietjet\b": "越捷航空",
        r"\bMandarin Airlines\b": "華信航空",
        r"\bShanghai Airlines\b": "上海航空",
        r"\bAir New Zealand\b": "紐西蘭航空",
        r"\bCebu Pacific\b": "宿霧太平洋航空",
        r"\bSichuan Airlines\b": "四川航空",
        r"\bVirgin Australia\b": "維珍澳洲航空",
        r"\bAir China\b": "中國國際航空"
    }

    # 處理航空公司名稱
    def replace_airlines(airline):
        for pattern, replacement in airline_mapping.items():
            airline = re.sub(pattern, replacement, airline, flags=re.IGNORECASE)
        return airline

    merged_data['航空公司'] = merged_data['航空公司'].apply(replace_airlines)

    # 按條件分組（出發日期、航班代碼、機型）
    grouped = merged_data.groupby(['出發日期', '航班代碼', '機型'], as_index=False).agg(
        平均價格=('價格', 'mean'),
        最低價格=('價格', 'min'),
        中位數價格=('價格', 'median')
    )
    grouped['平均價格'] = grouped['平均價格'].round().astype(int)

    # 計算最低價格的剩餘天數
    min_price_days = merged_data.loc[
        merged_data.groupby(['出發日期', '航班代碼', '機型'])['價格'].idxmin(),
        ['出發日期', '航班代碼', '機型', 'day left']
    ]
    min_price_days.rename(columns={'day left': '最低價格剩餘天數'}, inplace=True)

    # 合併分組後的統計數據
    final_data = pd.merge(merged_data.drop(columns=['價格', 'day left']),
                          grouped, on=['出發日期', '航班代碼', '機型'], how='inner')
    final_data = pd.merge(final_data, min_price_days, on=['出發日期', '航班代碼', '機型'], how='inner')

    # 去除重複行，只保留同一條航班的第一筆資料
    final_data = final_data.drop_duplicates(subset=['出發日期', '航班代碼', '機型'])

    # 排序結果
    final_data = final_data.sort_values(by=['出發日期', '航班代碼']).reset_index(drop=True)

    # 儲存結果
    final_data.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"處理完成，結果已按出發日期與航班代碼排序，儲存到 {output_path}")


# 主程式執行流程
base_path = '/Users/yuchingchen/Documents/專題/data'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/sydney.csv'

# 執行合併與處理
process_and_merge_files('1021', '1119', base_path, output_path)

處理完成，結果已按出發日期與航班代碼排序，儲存到 /Users/yuchingchen/Documents/專題/cleaned_data/sydney.csv
